# PREPROCESSING FOR NN
Une fois qu'on a retirer le data via le package TCGAbiolinks et utilise le SummarizedExperiment et on charge la data dans python 

Importation et définition du dossier de sortie: OUTDIR

Checker qu’on a tout ce qu’on avait dans GDCprepare

In [150]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

OUTDIR = "./tcga_brca_qc_report"
os.makedirs(OUTDIR, exist_ok=True)

RANDOM_STATE = 42

In [151]:
counts = pd.read_csv("brca_counts.csv", index_col=0) #(gene x echantillon)
meta   = pd.read_csv("brca_metadata.csv", index_col=0) #(echantillon x variable)
genes  = pd.read_csv("brca_genes.csv", index_col=0)

In [152]:
print(counts.shape)
print(meta.shape)
assert counts.shape[1] == meta.shape[0]

# garder uniquement les primary tumors
mask = (meta["sample_type"] == "Primary Tumor").to_numpy()

counts = counts.loc[:, mask]
meta   = meta.loc[mask].reset_index(drop=True)

(60660, 1224)
(1224, 90)


ON A LES MEMES RESULTATS QUE SUR R donc c'est bon !
Maintenant on check la metadata

In [153]:
#To check all the data
meta.columns.sort_values()

#To check if a column exists in the dataframe
if 'sample' in meta.columns:
    print("there is sample")
else:
    print("no sample found")

there is sample


# VERIFICATION INITIALES
## VERIFIER LES OBJETS EN MEMOIRE

In [154]:
# Attendu: counts, genes, meta existent déjà (issus de GDCprepare)
print(type(counts), counts.shape)
print(type(genes), genes.shape)
print(type(meta), meta.shape)

display(counts.iloc[:5, :5])
display(meta.head())
display(genes.head())

<class 'pandas.core.frame.DataFrame'> (60660, 1111)
<class 'pandas.core.frame.DataFrame'> (60660, 10)
<class 'pandas.core.frame.DataFrame'> (1111, 90)


,TCGA-BH-A18H-01A-11R-A12D-07,TCGA-E2-A14P-01A-31R-A12D-07,TCGA-AN-A04A-01A-21R-A034-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-AQ-A0Y5-01A-11R-A14M-07
ENSG00000000003.15,426,1033,1417,929,879
ENSG00000000005.6,3,21,9,3,9
ENSG00000000419.13,1439,4553,1982,933,2623
ENSG00000000457.14,1049,932,1227,802,1727
ENSG00000000460.17,326,463,284,143,421


,barcode,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,sample_id,pathology_report_uuid,submitter_id,...,paper_tobacco_smoking_history,paper_CNV.Clusters,paper_Mutation.Clusters,paper_DNA.Methylation.Clusters,paper_mRNA.Clusters,paper_miRNA.Clusters,paper_lncRNA.Clusters,paper_Protein.Clusters,paper_PARADIGM.Clusters,paper_Pan.Gyn.Clusters
0,TCGA-BH-A18H-01A-11R-A12D-07,TCGA-BH-A18H,TCGA-BH-A18H-01A,TP,Primary solid Tumor,TCGA-BH-A18H-01A,Primary,9321161b-d568-4d38-aed3-956786a06329,DF536409-601F-489B-9F8A-A2B372BF9999,TCGA-BH-A18H,...,NaN,C4,C4,C1,C2,C3,C1,C1,C6,C3
1,TCGA-E2-A14P-01A-31R-A12D-07,TCGA-E2-A14P,TCGA-E2-A14P-01A,TP,Primary solid Tumor,TCGA-E2-A14P-01A,Primary,58bfe278-a80f-4722-b286-4d966214d244,FD8CDFD9-6DCF-4CE1-9BB1-A7FF568EE53C,TCGA-E2-A14P,...,NaN,C4,C4,C1,C2,C7,C2,C1,C4,C3
2,TCGA-AN-A04A-01A-21R-A034-07,TCGA-AN-A04A,TCGA-AN-A04A-01A,TP,Primary solid Tumor,TCGA-AN-A04A-01A,Primary,1d8f72b7-8596-40e8-afdf-244dd897758c,6FB7A794-C3E7-4501-A844-CF65B9C3D32C,TCGA-AN-A04A,...,NaN,C6,C1,C1,C1,C3,C2,C1,C5,C5
3,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5L-AAT1,TCGA-5L-AAT1-01A,TP,Primary solid Tumor,TCGA-5L-AAT1-01A,Primary,a31dec9b-3626-4cb0-8265-1425d5ce1c1e,B5CA42BB-9514-42C6-9FB0-C8889C1DC51A,TCGA-5L-AAT1,...,NaN,NaN,C9,C1,C2,NaN,NaN,C2,NaN,C1
4,TCGA-AQ-A0Y5-01A-11R-A14M-07,TCGA-AQ-A0Y5,TCGA-AQ-A0Y5-01A,TP,Primary solid Tumor,TCGA-AQ-A0Y5-01A,Primary,f5675b71-ed1b-4fdb-be07-d627dc98ed88,5BC01981-366E-44EC-A986-087DB7904E5C,TCGA-AQ-A0Y5,...,NaN,C4,C4,C2,C1,C2,NaN,NaN,C5,NaN


,source,type,score,phase,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene
ENSG00000000003.15,HAVANA,gene,NaN,NaN,ENSG00000000003.15,protein_coding,TSPAN6,2,HGNC:11858,OTTHUMG00000022002.2
ENSG00000000005.6,HAVANA,gene,NaN,NaN,ENSG00000000005.6,protein_coding,TNMD,2,HGNC:17757,OTTHUMG00000022001.2
ENSG00000000419.13,HAVANA,gene,NaN,NaN,ENSG00000000419.13,protein_coding,DPM1,2,HGNC:3005,OTTHUMG00000032742.2
ENSG00000000457.14,HAVANA,gene,NaN,NaN,ENSG00000000457.14,protein_coding,SCYL3,2,HGNC:19285,OTTHUMG00000035941.6
ENSG00000000460.17,HAVANA,gene,NaN,NaN,ENSG00000000460.17,protein_coding,C1orf112,2,HGNC:25565,OTTHUMG00000035821.9


## DEFINIR LES COLONNES CLES 
On défini sample_col et patient_col car TCGA a plusieurs niveaux d’identifiants et on veut être sûr qu’on a le bon

In [155]:
SAMPLE_COL  = "barcode"   # colonne meta qui correspond aux colonnes de counts
PATIENT_COL = "patient"  # colonne meta pour grouper les échantillons par patient

if SAMPLE_COL not in meta.columns:
    raise ValueError(f"{SAMPLE_COL} absent de meta.columns. Colonnes disponibles: {list(meta.columns)[:30]} ...")

if PATIENT_COL not in meta.columns:
    raise ValueError(f"{PATIENT_COL} absent de meta.columns. Colonnes disponibles: {list(meta.columns)[:30]} ...")

## ALIGNER LES COUNTS ET LES META 
Réordonne/Aligne: intersection entre counts.columns et meta[SAMPLE_COL]

Pour que chaque colonne de counts correspondent à une ligne de meta 

In [156]:
# counts: genes x samples
if counts.index.duplicated().any():
    raise ValueError("counts.index contient des doublons (gènes). À corriger avant QC.")

if counts.columns.duplicated().any():
    raise ValueError("counts.columns contient des doublons (échantillons). À corriger avant QC.")

# Intersection
meta_samples = set(meta[SAMPLE_COL].astype(str))
common = [s for s in counts.columns.astype(str) if s in meta_samples]

if len(common) < 10:
    raise ValueError("Trop peu d'échantillons en commun entre counts et meta. Vérifie les IDs.")

counts = counts.loc[:, common].copy()

# Meta réordonnée selon counts
meta = meta.copy()
meta[SAMPLE_COL] = meta[SAMPLE_COL].astype(str)
meta = meta[meta[SAMPLE_COL].isin(common)].set_index(SAMPLE_COL).loc[common].reset_index()

print("Aligned counts:", counts.shape)
print("Aligned meta:", meta.shape)

Aligned counts: (60660, 1111)
Aligned meta: (1111, 90)


## VERIFIER LA COHERENCE AVEC LES GENES
On vérifie si genes.index == counts.index → chaque ligne de counts correspond bien à un gène défini

In [157]:
# On suppose que genes a une colonne qui correspond à counts.index (ex: gene_id / ensembl_gene_id)
# Si gene est déjà indexé par le bon ID, laisser comme ça.
if genes.index.equals(counts.index):
    print("OK: genes.index == counts.index")
else:
    # sinon, mapper une colonne
    # -> adapter GENE_ID_COL si nécessaire -> pas besoin ici gene.index == counts.index
    GENE_ID_COL = None  # ex: "gene_id" ou "ensembl_gene_id"
    if GENE_ID_COL is None:
        print("NOTE: genes.index ne matche pas counts.index. Si tu veux joindre l'annotation, définis GENE_ID_COL.")
    else:
        if GENE_ID_COL not in genes.columns:
            raise ValueError(f"{GENE_ID_COL} absent de genes.columns.")
        genes = genes.set_index(GENE_ID_COL)
        genes = genes.loc[counts.index]
        print("genes réaligné sur counts.index:", genes.shape)

OK: genes.index == counts.index


# SPLIT PAR GROUPE PATIENT
On split 80/20 avc GroupeShuffleSplit, on groupe par patient_id et on vérifie qu’aucun patient n’est dans les deux sets

In [158]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)

idx = np.arange(meta.shape[0])
groups = meta[PATIENT_COL].astype(str).values

train_idx, test_idx = next(gss.split(idx, groups=groups))

train_samples = meta.loc[train_idx, SAMPLE_COL].tolist()
test_samples  = meta.loc[test_idx,  SAMPLE_COL].tolist()

# Sanity check: pas de patient en commun
train_pat = set(meta.loc[train_idx, PATIENT_COL].astype(str))
test_pat  = set(meta.loc[test_idx,  PATIENT_COL].astype(str))
overlap = train_pat & test_pat
if overlap:
    raise RuntimeError(f"Leakage patient détecté: {len(overlap)} patients se retrouvent dans train et test.")

counts_tr = counts[train_samples]
counts_te = counts[test_samples]

meta_tr = meta.set_index(SAMPLE_COL).loc[train_samples].reset_index()
meta_te = meta.set_index(SAMPLE_COL).loc[test_samples].reset_index()

print("Train:", counts_tr.shape, "Test:", counts_te.shape)

Train: (60660, 889) Test: (60660, 222)


# QC Calcul des métrique QC
On calcul, pour chaque sample:

- *libsize*: somme des counts
- *detected_genes*: nbr de gène exprimés
- *zero_fraction:* proportion de zéros
- *top1_frac*: dominance du gène le plus exprimé, top1 est une coordonnée PCA par échantillon.

Pour qu’avec ces Qc on détecte les échantillons de mauvaise qualité, problème technique et les outliers évidents

In [159]:
def sample_qc_metrics(counts_gxs: pd.DataFrame) -> pd.DataFrame:
    libsize = counts_gxs.sum(axis=0).astype(float)
    detected = (counts_gxs > 0).sum(axis=0).astype(float)
    zero_frac = (counts_gxs == 0).mean(axis=0).astype(float)
    top1 = (counts_gxs.max(axis=0) / libsize.replace(0, np.nan)).fillna(0.0).astype(float)
    return pd.DataFrame({
        "libsize": libsize,
        "detected_genes": detected,
        "zero_fraction": zero_frac,
        "top1_frac": top1,
    })

qc_tr = sample_qc_metrics(counts_tr)
qc_te = sample_qc_metrics(counts_te)

qc_tr = qc_tr.join(meta_tr.set_index(SAMPLE_COL), how="left")
qc_te = qc_te.join(meta_te.set_index(SAMPLE_COL), how="left")

display(qc_tr.head())

,libsize,detected_genes,zero_fraction,top1_frac,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,...,paper_tobacco_smoking_history,paper_CNV.Clusters,paper_Mutation.Clusters,paper_DNA.Methylation.Clusters,paper_mRNA.Clusters,paper_miRNA.Clusters,paper_lncRNA.Clusters,paper_Protein.Clusters,paper_PARADIGM.Clusters,paper_Pan.Gyn.Clusters
TCGA-BH-A18H-01A-11R-A12D-07,41748828.0,31328.0,0.483548,0.016142,TCGA-BH-A18H,TCGA-BH-A18H-01A,TP,Primary solid Tumor,TCGA-BH-A18H-01A,Primary,...,NaN,C4,C4,C1,C2,C3,C1,C1,C6,C3
TCGA-AN-A04A-01A-21R-A034-07,68281533.0,30936.0,0.490010,0.017330,TCGA-AN-A04A,TCGA-AN-A04A-01A,TP,Primary solid Tumor,TCGA-AN-A04A-01A,Primary,...,NaN,C6,C1,C1,C1,C3,C2,C1,C5,C5
TCGA-5L-AAT1-01A-12R-A41B-07,24260663.0,29142.0,0.519585,0.014799,TCGA-5L-AAT1,TCGA-5L-AAT1-01A,TP,Primary solid Tumor,TCGA-5L-AAT1-01A,Primary,...,NaN,NaN,C9,C1,C2,NaN,NaN,C2,NaN,C1
TCGA-AQ-A0Y5-01A-11R-A14M-07,63988272.0,32629.0,0.462100,0.012445,TCGA-AQ-A0Y5,TCGA-AQ-A0Y5-01A,TP,Primary solid Tumor,TCGA-AQ-A0Y5-01A,Primary,...,NaN,C4,C4,C2,C1,C2,NaN,NaN,C5,NaN
TCGA-E9-A1R4-01A-21R-A14D-07,50077273.0,33222.0,0.452324,0.013934,TCGA-E9-A1R4,TCGA-E9-A1R4-01A,TP,Primary solid Tumor,TCGA-E9-A1R4-01A,Primary,...,NaN,C6,C9,C1,C1,C3,C1,C1,C6,C3


# QC- Seuil appris sur Train
## Fonction seuil robuste
On définit median ± k*MAD (*Median Absolute Deviation)*

Parce que les distribution RNA-seq ne sont pas normales, MAD est robustes aux outliers

In [160]:
def mad_bounds(x: pd.Series, k: float = 3.5):
    x = pd.to_numeric(x, errors="coerce").dropna().astype(float)
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    robust_sd = 1.4826 * mad if mad > 0 else np.std(x)
    lo = med - k * robust_sd
    hi = med + k * robust_sd
    return lo, hi

## Apprentissage/définition des seuils QC sur train ( à faire sur test après)
On calcul les seuils QC uniqmt sur train, on ajoute un plancher (garde fou biologique)

Cela évite toute information pour test, les seuils deviennent des hyper-paramètre appris

Bulk RNA-seq humain :
~15 000–20 000 gènes détectés

In [161]:
lib_lo, lib_hi = mad_bounds(qc_tr["libsize"], k=3.5)
det_lo, det_hi = mad_bounds(qc_tr["detected_genes"], k=3.5) #nbr de gènes détectés par échantillon
zf_lo,  zf_hi  = mad_bounds(qc_tr["zero_fraction"], k=3.5)
t1_lo,  t1_hi  = mad_bounds(qc_tr["top1_frac"], k=3.5) #top1 = projection de chaque échantillon sur la 1ʳᵉ composante principale (PC1).

# Option: planchers/plafonds de sécurité (à ajuster selon dataset)
lib_lo = max(lib_lo, 1e6)     # On veut au moins 1 million de lectures/read par échantillon (en dessous on a du bruit technique dominant)
det_lo = max(det_lo, 10000)    # On elimine les échantillons avec moins de 10000 gènes détectés, car grand dataset (+1000 échantillons)
zf_hi  = min(zf_hi, 0.95)       # on autorise au maximum 5 % des échantillons les plus extrêmes, quelques outliers abbérants peuvent gonfler artificiellement le seuils oui annuler l'effet de QC.

qc_thresholds = {
    "libsize_lo": lib_lo, "libsize_hi": lib_hi,
    "detected_lo": det_lo, "detected_hi": det_hi,
    "zero_lo": zf_lo, "zero_hi": zf_hi,
    "top1_lo": t1_lo, "top1_hi": t1_hi
}

qc_thresholds

{'libsize_lo': 3643916.172600001,
 'libsize_hi': 111782829.8274,
 'detected_lo': 25803.17,
 'detected_hi': 39294.83,
 'zero_lo': 0.3522118364655456,
 'zero_hi': 0.5746262776129245,
 'top1_lo': -0.009222292377061271,
 'top1_hi': 0.03635283312095532}

## Application des seuils au Test dataset
ON applique ces mêmes seuil au test, les quelques samples exclus sont gardé dans la colonnes crée keep.

In [162]:
def apply_qc(df_qc: pd.DataFrame, thr: dict) -> pd.Series:
    m = pd.Series(True, index=df_qc.index)
    m &= df_qc["libsize"].between(thr["libsize_lo"], thr["libsize_hi"])
    m &= df_qc["detected_genes"].between(thr["detected_lo"], thr["detected_hi"])
    m &= df_qc["zero_fraction"].between(thr["zero_lo"], thr["zero_hi"])
    m &= df_qc["top1_frac"].between(thr["top1_lo"], thr["top1_hi"])
    return m

keep_tr = apply_qc(qc_tr, qc_thresholds)
keep_te = apply_qc(qc_te, qc_thresholds)

print("Train kept:", int(keep_tr.sum()), "/", len(keep_tr))
print("Test  kept:", int(keep_te.sum()), "/", len(keep_te))

qc_tr_out = qc_tr.copy()
qc_te_out = qc_te.copy()
qc_tr_out["keep"] = keep_tr.values
qc_te_out["keep"] = keep_te.values

Train kept: 814 / 889
Test  kept: 208 / 222


# GRAPHS POUR QC
## SCATTER + HIST + LIGENS SEUIL
Ce sont les fonctions pour histogrames, scatterplots et sauvegarde automatique.

In [163]:
def savefig(name):
    path = os.path.join(OUTDIR, name)
    plt.tight_layout()
    plt.savefig(path, dpi=160, bbox_inches="tight")
    plt.close()
    return path

def qc_hist(train_df, test_df, col, thr=None, bins=60):
    plt.figure()
    plt.hist(train_df[col].dropna(), bins=bins, alpha=0.6, label="train")
    plt.hist(test_df[col].dropna(), bins=bins, alpha=0.6, label="test")
    plt.xlabel(col); plt.ylabel("n")
    if thr is not None:
        lo, hi = thr
        plt.axvline(lo)
        plt.axvline(hi)
    plt.legend(frameon=False)
    return savefig(f"qc_hist_{col}.png")

def qc_scatter(df, x, y, keep_col="keep", title=""):
    plt.figure()
    if keep_col in df.columns:
        m = df[keep_col].astype(bool).values
        plt.scatter(df.loc[m, x], df.loc[m, y], s=12, alpha=0.7, label="kept")
        plt.scatter(df.loc[~m, x], df.loc[~m, y], s=12, alpha=0.7, label="filtered")
        plt.legend(frameon=False)
    else:
        plt.scatter(df[x], df[y], s=12, alpha=0.7)
    plt.xlabel(x); plt.ylabel(y); plt.title(title)
    return savefig(f"qc_scatter_{x}_vs_{y}_{title.replace(' ', '_')}.png")

## QC 1, 2, 3, 4 : LEURS DISTRIBUTION ET SEUIL
Histograme train vs test et ajout des lignes de seuil

Pour vérifier que les seuils sont raisonnables et que train & test ont des distribution comparable

In [164]:
qc_hist(qc_tr_out, qc_te_out, "libsize", thr=(qc_thresholds["libsize_lo"], qc_thresholds["libsize_hi"]))
qc_hist(qc_tr_out, qc_te_out, "detected_genes", thr=(qc_thresholds["detected_lo"], qc_thresholds["detected_hi"]))
qc_hist(qc_tr_out, qc_te_out, "zero_fraction", thr=(qc_thresholds["zero_lo"], qc_thresholds["zero_hi"]))
qc_hist(qc_tr_out, qc_te_out, "top1_frac", thr=(qc_thresholds["top1_lo"], qc_thresholds["top1_hi"]))

display(qc_tr_out[["libsize","detected_genes","zero_fraction","top1_frac","keep"]].describe())

,libsize,detected_genes,zero_fraction,top1_frac
count,8.890000e+02,889.000000,889.000000,889.000000
mean,5.771642e+07,32537.221597,0.463613,0.017842
std,1.466917e+07,2529.492654,0.041700,0.013975
min,1.922512e+07,24322.000000,0.098566,0.004829
25%,4.734575e+07,31190.000000,0.442598,0.009936
50%,5.771337e+07,32549.000000,0.463419,0.013565
75%,6.817859e+07,33812.000000,0.485823,0.019978
max,1.140157e+08,54681.000000,0.599044,0.105110


## QC 4 SCATTER POUR LES OUTLIERS
libsize vs detected_genes, les couleurs : conversé vs filtré

Pour visualisé les outliers rapidement:
pour chaque échantillons on a en x-axis, le nombre total de read RNA-seq, le nombre de gènes avec counts > 0
-> chaque point est un échantillon.
La relation attendue est plus au moins linéaire et croissante <=> plus on séquence plus on a de counts.

In [165]:
qc_scatter(qc_tr_out, "libsize", "detected_genes", title="train")
qc_scatter(qc_te_out, "libsize", "detected_genes", title="test")

'./tcga_brca_qc_report/qc_scatter_libsize_vs_detected_genes_test.png'

# FILTRAGE GENE ET NORMALISATION 
## FILTRER LES ECHANTILLONS APRES QC
On supprime les samples rejetés par QC, on met à jour counts et meta

On peut donc travaillé sur des données propres

In [166]:
train_kept = qc_tr_out.index[qc_tr_out["keep"]].tolist()
test_kept  = qc_te_out.index[qc_te_out["keep"]].tolist()

counts_tr_f = counts_tr[train_kept]
counts_te_f = counts_te[test_kept]

meta_tr_f = meta_tr.set_index(SAMPLE_COL).loc[train_kept].reset_index()
meta_te_f = meta_te.set_index(SAMPLE_COL).loc[test_kept].reset_index()

print(counts_tr_f.shape, counts_te_f.shape)

(60660, 814) (60660, 208)


## FILTRER GENES SUR TRAIN 
On garde les gènes exprimés dans au moins ≥ X% des samples train

Pour supprimer le bruit et réduire la dimension après

In [167]:
min_frac = 0.10
min_n = max(5, int(np.ceil(min_frac * counts_tr_f.shape[1])))

gene_keep = (counts_tr_f > 0).sum(axis=1) >= min_n

counts_tr_fg = counts_tr_f.loc[gene_keep].copy()
counts_te_fg = counts_te_f.loc[gene_keep].copy()

print("Genes kept:", int(gene_keep.sum()), "sur", counts_tr_f.shape[0], "totaux")

Genes kept: 43638 sur 60660 totaux


# PCA sur train uniquemt
## sélection des top variant genes sur train
On fait la variance calculé sur le train et on sélectionne les top gènes

Pour avoir une réduction dimensionnelle biologiqmt informée

On fait une sélection des top 10k genes, et on ajoute un bloc pour l'exportation et analyse de ces gènes.

In [168]:
top_genes = min(10000, counts_tr_fg.shape[0])
gene_var = counts_tr_fg.var(axis=1).sort_values(ascending=False)
sel_genes = gene_var.head(top_genes).index

Xtr_sel = counts_tr_fg.loc[sel_genes].T  # samples x genes
Xte_sel = counts_te_fg.loc[sel_genes].T

print(Xtr_sel.shape, Xte_sel.shape)

(814, 10000) (208, 10000)


# Exporter des HVG (10k) protein_coding


In [169]:
df_pc = gene_var.reset_index()
df_pc.columns = ["ensembl_id", "variance"]

df_pc = df_pc.merge(
    genes[["gene_name", "gene_type"]].reset_index().rename(columns={"index": "ensembl_id"}),
    on="ensembl_id",
    how="left"
)

TOP_N_PC = 300   # pour l'exportation, on cherche: Quels sont les gènes protein_coding les plus variables du dataset ?
df_pc_top = df_pc.head(TOP_N_PC)

df_pc_top.to_csv(
    os.path.join(OUTDIR, f"hvg_top_{TOP_N_PC}_protein_coding.csv"),
    index=False
)

gene_list_pc = df_pc_top["gene_name"].fillna(df_pc_top["ensembl_id"]).astype(str).tolist()
with open(os.path.join(OUTDIR, f"hvg_top_{TOP_N_PC}_protein_coding_gene_list.txt"), "w") as f:
    f.write("\n".join(gene_list_pc))

print("Saved:",
      f"hvg_top_{TOP_N_PC}_protein_coding.csv",
      f"hvg_top_{TOP_N_PC}_protein_coding_gene_list.txt")

Saved: hvg_top_300_protein_coding.csv hvg_top_300_protein_coding_gene_list.txt


## Standardisation et PCA en fit sur train et transform sur Test
On fait un StandardScaler et PCA sur le train.

In [170]:
scaler = StandardScaler()
Xtr_z = scaler.fit_transform(Xtr_sel)
Xte_z = scaler.transform(Xte_sel)

pca = PCA(n_components=20, random_state=RANDOM_STATE)
Ztr = pca.fit_transform(Xtr_z)
Zte = pca.transform(Xte_z)

expl = pca.explained_variance_ratio_
print("Explained variance PC1-5:", expl[:5], " | sum PC1-20:", expl[:20].sum())
print("Ztr:", Ztr.shape)
print("Zte:", Zte.shape)

Explained variance PC1-5: [0.18980305 0.06868443 0.05816674 0.03684524 0.02278491]  | sum PC1-20: 0.5124001669094745
Ztr: (814, 20)
Zte: (208, 20)


## Les plots
On fait une PCA avec PC1 vs PC2 et train vs test

Pour vérifier la structure globale, le batch effect et les outliers persistant

In [171]:
plt.figure()
plt.scatter(Ztr[:,0], Ztr[:,1], s=14, alpha=0.7, label="train")
plt.scatter(Zte[:,0], Zte[:,1], s=14, alpha=0.7, label="test")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend(frameon=False)
savefig("pca_train_vs_test.png")

'./tcga_brca_qc_report/pca_train_vs_test.png'

# METADATA
## SELECTION AUTO DES COLONNES CATEGORIELLES ET NUMERIQUES

In [172]:
def pick_categorical_cols(df, max_unique=30):
    cols = []
    for c in df.columns:
        if c in [SAMPLE_COL, PATIENT_COL]:
            continue
        nun = df[c].nunique(dropna=True)
        if nun >= 2 and nun <= max_unique and df[c].dtype == "object":
            cols.append(c)
    return cols

def pick_numeric_cols(df):
    cols = []
    for c in df.columns:
        if c in [SAMPLE_COL, PATIENT_COL]:
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            cols.append(c)
    return cols

cat_cols = pick_categorical_cols(meta_tr_f, max_unique=25)
num_cols = pick_numeric_cols(meta_tr_f)

print("Categorical candidates:", cat_cols)
print("Numeric candidates:", num_cols)

Categorical candidates: ['specimen_type', 'preservation_method', 'synchronous_malignancy', 'ajcc_pathologic_stage', 'laterality', 'tissue_or_organ_of_origin', 'prior_malignancy', 'prior_treatment', 'diagnosis_is_primary_disease', 'method_of_diagnosis', 'ajcc_staging_system_edition', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m', 'classification_of_tumor', 'icd_10_code', 'site_of_resection_or_biopsy', 'metastasis_at_diagnosis', 'follow_ups_disease_response', 'race', 'gender', 'ethnicity', 'vital_status', 'age_is_obfuscated', 'country_of_residence_at_enrollment', 'paper_Included_in_previous_marker_papers', 'paper_vital_status', 'paper_pathologic_stage', 'paper_BRCA_Pathology', 'paper_BRCA_Subtype_PAM50', 'paper_CNV.Clusters', 'paper_Mutation.Clusters', 'paper_DNA.Methylation.Clusters', 'paper_mRNA.Clusters', 'paper_miRNA.Clusters', 'paper_lncRNA.Clusters', 'paper_Protein.Clusters', 'paper_PARADIGM.Clusters', 'paper_Pan.Gyn.Clusters']
Numeric candidates: ['days_to_collecti

## BARPLOT (CATEGO) ET HISTO (NUMERIC)
### uncomment plt show to see the figures directly (carefull, there are many of them)

In [173]:
def barplot_top(df_tr, df_te, col, k=15):
    tr = df_tr[col].astype(str).fillna("NA").value_counts().head(k)
    te = df_te[col].astype(str).fillna("NA").value_counts().head(k)

    cats = sorted(set(tr.index) | set(te.index))
    trv = [tr.get(c, 0) for c in cats]
    tev = [te.get(c, 0) for c in cats]

    x = np.arange(len(cats))
    plt.figure(figsize=(max(6, len(cats)*0.5), 4))
    plt.bar(x-0.2, trv, width=0.4, label="train")
    plt.bar(x+0.2, tev, width=0.4, label="test")
    plt.xticks(x, cats, rotation=45, ha="right")
    plt.ylabel("n"); plt.title(col)
    plt.legend(frameon=False)
    #plt.show()
    return savefig(f"meta_bar_{col}.png")

def hist_numeric(train_df, test_df, col, bins=60):
    plt.figure()
    plt.hist(pd.to_numeric(train_df[col], errors="coerce").dropna(), bins=bins, alpha=0.6, label="train")
    plt.hist(pd.to_numeric(test_df[col], errors="coerce").dropna(), bins=bins, alpha=0.6, label="test")
    plt.xlabel(col); plt.ylabel("n")
    plt.legend(frameon=False)
    #plt.show()
    return savefig(f"meta_hist_{col}.png")

# Plots (limite volontaire pour éviter 200 figures)
for c in cat_cols[:8]:
    barplot_top(meta_tr_f, meta_te_f, c)

for c in num_cols[:8]:
    hist_numeric(meta_tr_f, meta_te_f, c)

# SAUVEGARDE

In [174]:
qc_tr_out.to_csv(os.path.join(OUTDIR, "qc_train.csv"))
qc_te_out.to_csv(os.path.join(OUTDIR, "qc_test.csv"))
pd.Series(qc_thresholds).to_csv(os.path.join(OUTDIR, "qc_thresholds_train_derived.csv"))

meta_tr_f.to_csv(os.path.join(OUTDIR, "meta_train_postQC.csv"), index=False)
meta_te_f.to_csv(os.path.join(OUTDIR, "meta_test_postQC.csv"), index=False)

# Matrices prêtes ML 
Xtr_sel.to_csv(os.path.join(OUTDIR, "X_train_logCPM_selectedGenes.csv"))
Xte_sel.to_csv(os.path.join(OUTDIR, "X_test_logCPM_selectedGenes.csv"))

np.save(os.path.join(OUTDIR, "X_train_z.npy"), Xtr_z)
np.save(os.path.join(OUTDIR, "X_test_z.npy"), Xte_z)


# Extraire les vrais noms de gènes depuis sel_genes (ensembl IDs) et les mapper aux gene_name dans brca_genes.csv
# sel_genes contient les ensembl IDs des 10,000 gènes sélectionnés
selected_gene_names = []

for ensembl_id in sel_genes:
    # Chercher le gene_name correspondant dans genes DataFrame
    if ensembl_id in genes.index:
        gene_name = genes.loc[ensembl_id, 'gene_name']
        # Si gene_name est NaN, utiliser l'ensembl_id
        if pd.isna(gene_name):
            selected_gene_names.append(ensembl_id)
        else:
            selected_gene_names.append(str(gene_name))
    else:
        # Si pas trouvé, utiliser l'ensembl_id
        selected_gene_names.append(ensembl_id)

# Convertir en array numpy
gene_names_array = np.array(selected_gene_names, dtype=object)

# Sauvegarder
np.save(os.path.join(OUTDIR, "gene_names.npy"), gene_names_array)

print(f"✓ Saved {len(gene_names_array)} gene names to gene_names.npy")
print(f"  First 10 genes: {gene_names_array[:10]}")
print(f"  Example genes: {gene_names_array[[0, 100, 500, 1000, 5000]]}")

print("Saved to:", OUTDIR)

✓ Saved 10000 gene names to gene_names.npy
  First 10 genes: ['COL1A1' 'FN1' 'IGHG1' 'COL1A2' 'MT-CO1' 'CPB1' 'COL3A1' 'MT-RNR2'
 'MT-ND4' 'IGKC']
  Example genes: ['COL1A1' 'LUM' 'SLC7A5' 'RAN' 'TGFB1']
Saved to: ./tcga_brca_qc_report


# GO enrichment 
On le fais sur gene_type dans brca_genes.csv, car GO beaucoup interprétable sur des gènes codants, et es gènes très variables peuvent inclure des lncRNA/pseudogenes qui ajoutent du bruit pour GO

In [175]:
try:
    import gseapy as gp

    GO_LIBRARY = "GO_Biological_Process_2021"
    go_outdir = os.path.join(OUTDIR, "go_hvg_enrichr")
    os.makedirs(go_outdir, exist_ok=True)

    enr = gp.enrichr(
        gene_list=gene_list_pc,     
        gene_sets=[GO_LIBRARY],
        organism="Human",
        outdir=go_outdir,
        cutoff=0.05
    )

    if hasattr(enr, "results") and enr.results is not None:
        enr.results.to_csv(os.path.join(OUTDIR, "go_hvg_enrichment.csv"), index=False)
        print("Saved:", os.path.join(OUTDIR, "go_hvg_enrichment.csv"))
    else:
        print("GO enrichment returned no results.")

except Exception as e:
    print("GO enrichment skipped/failed:", repr(e))
    print("If no internet or gseapy not installed, this is expected.")

Saved: ./tcga_brca_qc_report/go_hvg_enrichment.csv
